# Zion Data Analysis Notebook

In [4]:
import os
from glob import glob
import time
from collections import UserDict, UserString
import numpy as np
import pandas as pd

from skimage.color import rgb2hsv
from skimage import filters, morphology, segmentation, measure
from scipy.optimize import nnls
import cv2

from matplotlib import pyplot as plt



Now give the notebook a directory of tiffs to work in:

In [5]:
session_tif_dirpath = ""

## Step 1: Spot Detection

In [6]:
#define parameters

#run roi detector, display result



## Step 2: Define color dictionary

Either provide a file OR select spot (by label) for each base:

In [7]:
# Run 